In [ ]:
# Pick directory --> load all data into study dict

# study --> A1, A2, ... --> A1S1, A1S2, ... A2S1, A2S2, ... --> A1S1C1, A1S1C2, ..., A1S2C1, ...

# sort cell data, match across waveforms

# batch run spikeAnalysisGUI (firing data) {batch_ratemaps.py}

# batch run spikeWidthGUI (spike width)

# batch run PowerSpectrum.vel --> save output LFP data and load into python

# batch run hfoGUI (LFP vizualization) 

# batch run PRISM/neurofunc (field properties) {batch_ratemaps.py}

# batch run odor/object analysis

In [1]:

# To be able to make edits to repo without having to restart notebook
%load_ext autoreload
%autoreload 2


In [2]:
# Outside imports
import os, sys
import numpy as np
import matplotlib.pyplot as plt

# Set necessary paths / make project path = ...../neuroscikit/
unit_matcher_path = os.getcwd()
prototype_path = os.path.abspath(os.path.join(unit_matcher_path, os.pardir))
project_path = os.path.abspath(os.path.join(prototype_path, os.pardir))
lab_path = os.path.abspath(os.path.join(project_path, os.pardir))
out_path = os.path.abspath(os.path.join(lab_path, os.pardir))
sys.path.append(project_path)
os.chdir(project_path)
print(project_path)

c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit


In [3]:
# Internal imports

# Read write modules
from x_io.rw.axona.batch_read import make_study
from _prototypes.unit_matcher.read_axona import read_sequential_sessions, temp_read_cut
from _prototypes.unit_matcher.write_axona import format_new_cut_file_name

# Unit matching modules
from _prototypes.unit_matcher.main import format_cut, run_unit_matcher, map_unit_matches_first_session, map_unit_matches_sequential_session
from _prototypes.unit_matcher.session import compare_sessions
from _prototypes.unit_matcher.waveform import time_index, derivative, derivative2, morphological_points

# External imports

import math
from sklearn import linear_model
from scipy import stats
import statsmodels.api as sm
from statsmodels.genmod.families import Poisson, Gaussian
from sklearn.metrics import r2_score
from statsmodels.genmod.families.links import identity, log


In [4]:
""" If a setting is not used for your analysis (e.g. smoothing_factor), just pass in an arbitrary value or pass in 'None' """
STUDY_SETTINGS = {

    'ppm': 511,  # EDIT HERE

    'smoothing_factor': None, # EDIT HERE

    'useMatchedCut': False,  # EDIT HERE, set to False if you want to use runUnitMatcher, set to True after to load in matched.cut file
}


# Switch devices to True/False based on what is used in the acquisition (to be extended for more devices in future)
device_settings = {'axona_led_tracker': True, 'implant': True} 

# Make sure implant metadata is correct, change if not, AT THE MINIMUM leave implant_type: tetrode
implant_settings = {'implant_type': 'tetrode', 'implant_geometry': 'square', 'wire_length': 25, 'wire_length_units': 'um', 'implant_units': 'uV'}

# WE ASSUME DEVICE AND IMPLANT SETTINGS ARE CONSISTENCE ACROSS SESSIONS

# Set channel count + add device/implant settings
SESSION_SETTINGS = {
    'channel_count': 4, # EDIT HERE, default is 4, you can change to other number but code will check how many tetrode files are present and set that to channel copunt regardless
    'devices': device_settings, # EDIT HERE
    'implant': implant_settings, # EDIT HERE
}

STUDY_SETTINGS['session'] = SESSION_SETTINGS

settings_dict = STUDY_SETTINGS

In [5]:
# data_dir = lab_path + r'\neuroscikit_test_data\20180530-ROUND-3300-1X2B3A' 
# data_dir = lab_path + r'\neuroscikit_test_data\LEC_odor\AD\Odor_119a-6' 
data_dir = lab_path + r'\neuroscikit_test_data\GusData\GAR' 


# To use in unit matching
settings_dict_unmatched = settings_dict
settings_dict_unmatched['useMatchedCut'] = False

In [ ]:
# unmatched_study = run_unit_matcher([data_dir], settings_dict_unmatched)
# print('COMPLETED UNIT MATCHING')

In [8]:
project_path

'c:\\Users\\aaoun\\OneDrive - cumc.columbia.edu\\Desktop\\HussainiLab\\neuroscikit'

In [6]:
import pickle 

for i in range(1,9,1):
    studies = {}
    settings_dict_unmatched['single_tet'] = i
    study = make_study([data_dir], settings_dict_unmatched)
    study.make_animals()

    studies[i] = study

    with open('dict_' + str(i) + '.pickle', 'wb') as handle:
        pickle.dump(studies, handle, protocol=pickle.HIGHEST_PROTOCOL)

SKIPPING, NO FILES FOUND FOR SESSION WITH SIGNATURE: c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\GusData\GAR/1_34/neurofunc_PRISM_1-34_20220203-12-50x50cm-1100um-Test1/Tetrode
SKIPPING, NO FILES FOUND FOR SESSION WITH SIGNATURE: c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\GusData\GAR/1a23/neurofunc_PRISM_1a23_20210824-13-50x50cm-1100um-Test1_temp/Tetrode
SKIPPING, NO FILES FOUND FOR SESSION WITH SIGNATURE: c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\GusData\GAR/1_25/neurofunc_PRISM_1-25_20210909-11-50x50cm-1200um-Test1_temp/Tetrode
SKIPPING, NO FILES FOUND FOR SESSION WITH SIGNATURE: c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\GusData\GAR/1_24/neurofunc_PRISM_1-24_20210824-13-50x50cm-1100um-Test1/Tetrode
SKIPPING, NO FILES FOUND FOR SESSION WITH SIGNATURE: c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\n

KeyboardInterrupt: 

In [7]:
study = make_study([data_dir], settings_dict_unmatched)
study.make_animals()

edu\Desktop\HussainiLab\neuroscikit_test_data\LEC_odor\AD\Odor_119a-6/ANT-119a-6_20170614-2450-odorxab 1 ['c:\\Users\\aaoun\\OneDrive - cumc', 'columbia', 'edu\\Desktop\\HussainiLab\\neuroscikit_test_data\\LEC_odor\\AD\\Odor_119a-6/ANT-119a-6_20170614-2450-odorxab', '1']
c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\LEC_odor\AD\Odor_119a-6/ANT-119a-6_20170614-2450-odorxab_1_matched.cut c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\LEC_odor\AD\Odor_119a-6/ANT-119a-6_20170614-2450-odorxab.1 c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\LEC_odor\AD\Odor_119a-6/ANT-119a-6_20170614-2450-odorxab.pos
1655
1655
1655
1655
Animal ID set
edu\Desktop\HussainiLab\neuroscikit_test_data\LEC_odor\AD\Odor_119a-6/ANT-119a-6_20170614-2450-odorxab 2 ['c:\\Users\\aaoun\\OneDrive - cumc', 'columbia', 'edu\\Desktop\\HussainiLab\\neuroscikit_test_data\\LEC_odor\\AD\\Odor_119a-6/ANT-119a-6_20

In [24]:
def event_times_to_count(spike_object):
    dt = 0.5
    new_time_index = np.arange(0,spike_object.time_index[-1],dt)
    ct, bins = np.histogram(spike_object.event_times, bins=new_time_index)
    # print(ct.shape, bins[:-1].shape)
    return ct, bins[:-1]

def aggregate_event_times_matched(study):
    agg_events = []
    agg_event_objects = []
    agg_events_binary = []
    prev_time_index = None
    for animal in study.animals:

        max_matched_cell_count = len(animal.sessions[sorted(list(animal.sessions.keys()))[-1]].get_cell_data()['cell_ensemble'].cells)
        print(max_matched_cell_count)
        for k in range(int(max_matched_cell_count)):
            cell_label = k + 1
            cell_events = []
            cell_event_objects = []
            cell_events_binary = []
            for i in range(len(list(animal.sessions.keys()))):
                seskey = 'session_' + str(i+1)
                ses = animal.sessions[seskey]
                ensemble = ses.get_cell_data()['cell_ensemble']
                if cell_label in ensemble.get_cell_label_dict():
                    cell = ensemble.get_cell_by_id(cell_label)
                    cell_events.append(cell.event_times)
                    cell_event_objects.append(cell)
                    ct, time_index = event_times_to_count(cell.cluster)
                    if prev_time_index is not None:
                        assert time_index.all() == prev_time_index.all()
                    cell_events_binary.append(ct)
                    prev_time_index = time_index
            agg_events.append(cell_events)
            agg_event_objects.append(cell_event_objects)
            agg_events_binary.append(cell_events_binary)

    return agg_events, np.array(agg_event_objects), np.array(agg_events_binary), np.array(time_index)

def aggregate_event_times(study):
    agg_events = []
    agg_event_objects = []
    agg_events_binary = []
    prev_time_index = None
    for animal in study.animals:
        for ses in animal.ensembles:
            for cell in animal.ensembles[ses].cells:
                agg_events.append(cell.event_times)
                agg_event_objects.append(cell)
                ct, time_index = event_times_to_count(cell.cluster)
                if prev_time_index is not None:
                    assert time_index.all() == prev_time_index.all()
                agg_events_binary.append(ct)
                # agg_time_index.append(time_index)
                prev_time_index = time_index
                
    return agg_events, np.array(agg_event_objects), np.array(agg_events_binary), np.array(time_index)

def split_test_train(data, percentage=0.70):
    
    idx = int(percentage * data.shape[1])

    train = data[:idx]
    test = data[idx:]

    return train, test

def split_endog_exog(y, X, p):
    idx = int(p * y.shape[0])

    trainY = y[:idx]
    testY = y[idx:]
    print(X.shape, y.shape, idx)
    trainX = X[:,:idx]
    testX = X[:,idx:]

    print(trainX.shape, trainY.shape, testX.shape, testY.shape)

    return trainX, trainY, testX, testY

def speed2D(x, y, t):
    """calculates an averaged/smoothed speed"""

    N = len(x)
    v = np.zeros((N, 1))

    for index in range(1, N-1):
        v[index] = np.sqrt((x[index + 1] - x[index - 1]) ** 2 + (y[index + 1] - y[index - 1]) ** 2) / (
        t[index + 1] - t[index - 1])

    v[0] = v[1]
    v[-1] = v[-2]

    return v


def get_head_direction(x: np.ndarray, y: np.ndarray) -> np.ndarray:

    '''
        Will compute the head direction angle of subject over experiemnt.
        Params:
            x, y (np.ndarray):
                Arrays of x and y coordinates.

        Returns:
            np.ndarray:
                angles
            --------
            angles:
                Array of head direction angles in radians, reflecting what heading the subject
                during the course of the sesison.
    '''

    last_point = [0,0]  # Keep track of the last x,y point
    last_angle = 0      # Keep track of the most previous computed angle
    angles = []         # Will accumulate angles as they are computed

    # Iterate over the x and y points
    for i in range(len(x)):

        # Grab the current point
        current_point = [float(x[i]), float(y[i])]

        # If the last point is the same as the current point
        if (last_point[0] == current_point[0]) and (last_point[1] == current_point[1]):
            # Retain the same angle
            angle = last_angle

        else:
            # Compute the arctan (i.e the angle formed by the line defined by both points
            # and the horizontal axis [range -180 to +180])
            # Uses the formula arctan( (y2-y1) / (x2-x1))
            Y = current_point[1] - last_point[1]
            X = current_point[0] - last_point[0]
            angle = math.atan2(Y,X) * (180/np.pi) # Convert to degrees

        # Append angle value to list
        angles.append(angle)

        # Update new angle and last_point values
        last_point[0] = current_point[0]
        last_point[1] = current_point[1]
        last_angle = angle

    # Scale angles between 0 to 360 rather than -180 to 180
    angles = np.array(angles)
    angles = (angles + 360) % 360

    # Convert to radians
    angles = angles * (np.pi/180)

    return angles


def get_pos_regressors(position_obj):
    x = position_obj.x
    y = position_obj.y 

    speed = speed2D(x, y, position_obj.time_index)

    direction = get_head_direction(x,y)

    return x, y, speed, direction
    # return x, y

def get_time_regressors(time_index, trial_start_times, trial_length):
    session_ramping = time_index

    dt = time_index[1] - time_index[0]

    trial_ramping = []
    for i in range(len(trial_start_times)):
        ramp = np.arange(0,trial_length, dt)
        trial_ramping = np.hstack((trial_ramping, ramp))

    if len(trial_ramping) < len(session_ramping):
        trial_ramping = np.hstack((trial_ramping, np.arange(0, session_ramping[-1]-trial_start_times[-1]-60+dt, dt)))

    return session_ramping, trial_ramping

def get_odor_regressors(time_index, odor_labels, odor_presentation_times, odor_window, trial_length):
    odor_pulses = []

    post_odor_window = []

    for i in range(len(odor_labels)):
        pulse = np.zeros(len(time_index))
        post_pulse = np.zeros(len(time_index))
        for j in range(len(odor_presentation_times)):
            # print(np.where(np.array(time_index) >=  float(odor_presentation_times[j] + odor_window))[0], np.where(np.array(time_index) < float(odor_presentation_times[j] + odor_window))[0])
            end_post = np.where(np.array(time_index) >=  float(odor_presentation_times[j] + odor_window))[0][0]
            start = np.where(np.array(time_index) >= float(odor_presentation_times[j]))[0][0]
            end = np.where(np.array(time_index) < float(odor_presentation_times[j]) + odor_window)[0][-1]

            pulse[start:end] = 1

            post_pulse[end : end_post] = 1
            
        odor_pulses.append(pulse)
        post_odor_window.append(post_pulse)

    return odor_pulses, post_odor_window

def adjust_bio_regressors_to_cts(bio_regs, len_cts):
    # new_regs = []
    # for animal_reg in regs:
    #     new_ses_regs = []

    #     for ses_reg in animal_reg:

    #         vars = []

    #         for reg in ses_reg:
    #             vars.append(reg[:len_cts])

    #         new_ses_regs.append(vars)

    #     new_regs.append(new_ses_regs)

    new_regs = bio_regs[:,:,:len_cts]

    return new_regs

def agg_synth_regressors(time_index):

    session_ramping, trial_ramping = get_time_regressors(time_index, [0,60,120,180], 60)

    odor_pulses, post_odor_window = get_odor_regressors(time_index, ['X','B', 'A', 'X'], [0,60,120,180], 10, 60)

    regs = np.vstack((session_ramping, trial_ramping, odor_pulses, post_odor_window))
    reg_labels = ['sessionT', 'trialT', 'odorX', 'odorB']

    return np.array(regs)

def agg_bio_regressors(study):
    # x, y, speed, direction = get_pos_regressors(position_obj)

    bio_regs = []

    for animal in study.animals:
        ses_bio_regs = []
        for ses in animal.sessions:
            position_obj = animal.sessions[ses].get_position_data()['position']

            x, y, speed, direction = get_pos_regressors(position_obj)
            # x, y = get_pos_regressors(position_obj)

            ses_reg = np.array([x.squeeze(), y.squeeze(), speed.squeeze(), direction.squeeze()])
            # ses_reg = np.array([x.squeeze(), y.squeeze()])

            ses_bio_regs.append(ses_reg)

        if len(animal.sessions) == 1:
            ses_reg = ses_bio_regs.pop()
            ses_reg = ses_reg.reshape((4,-1))
            ses_bio_regs.append(ses_reg)

        bio_regs.append(ses_bio_regs)

    return bio_regs

def check_collinearity(regs):
    for i in range(len(regs)):
        for j in range(len(regs)):
            r, p = stats.pearsonr(regs[i], regs[j])

            if r > 0.1 and i != j:
                print(r)
                print(i,j)

def normalize_regs(X):
    copy = np.zeros(X.shape)

    for i in range(len(X)):
        copy[i] = X[i] / np.linalg.norm(X[i])

    return copy

def fit_glm_sklearn(y, X):
    
    trainX, trainY, testX, testY = split_endog_exog(y, X, 0.7)

    # print(trainX.shape, trainY.shape, testX.shape, testY.shape)

    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.pipeline import make_pipeline
    from sklearn.metrics import mean_squared_error
    # intercept added to X already
    clf = linear_model.PoissonRegressor(fit_intercept=True)
    # clf = make_pipeline(StandardScaler(), linear_model.PoissonRegressor())

    clf.fit(trainX.T, trainY)

    train_pred = clf.predict(trainX.T)

    # train_score = clf.score(trainX.T, trainY)
    train_score = mean_squared_error(train_pred, trainY)

    pred = clf.predict(testX.T)


    # test_score = clf.score(testX.T, testY)
    test_score = mean_squared_error(testY, pred)

    # print(train_score, test_score)
    coeff = clf.coef_   

    intercept = clf.intercept_

    # pred = clf.predict(testX.T)

    res = {'intercept': intercept, 'coeff': coeff, 'test_score': test_score, 'train_score': train_score}

    res_data = {'pred': pred, 'clf': clf}

    inp = {'trainX': trainX, 'trainY': trainY, 'testX': testX, 'testY': testY}

    return res, res_data, inp

def fit_glm_statsmodel(y, X, regularized=False):

    avg = np.mean(y)

    intercept = np.ones(len(y)) * avg
    X = np.vstack((intercept, X))
    
    trainX, trainY, testX, testY = split_endog_exog(y, X, 0.5)

    print(trainX.shape, trainY.shape, testX.shape, testY.shape)

    X[X != X] = 0

    poisson_model = sm.GLM(trainY, trainX.T, family=Gaussian(log()))

    if regularized:
        poisson_result = poisson_model.fit_regularized(alpha=0.1)
        b = poisson_result.params.reshape(-1, 1)
        intercept = b[0]
        coeff = b[1:]
        # DE = 1 - (poisson_result.deviance)/poisson_result.null_deviance
        DE = None
        AIC = poisson_result.aic
        BIC = poisson_result.bic
        fitted_vals = poisson_result.fittedvalues
        chi2 = poisson_result.pearson_chi2
        pvalues = poisson_result.pvalues
        tvalues= poisson_result.tvalues

    else:
        poisson_result = poisson_model.fit()
        b = poisson_result.params.reshape(-1, 1)
        intercept = b[0]
        coeff = b[1:]
        DE = 1 - (poisson_result.deviance)/poisson_result.null_deviance
        AIC = poisson_result.aic
        BIC = poisson_result.bic
        fitted_vals = poisson_result.fittedvalues
        chi2 = poisson_result.pearson_chi2
        pvalues = poisson_result.pvalues
        tvalues= poisson_result.tvalues

    trainPred = poisson_result.predict(trainX.T)
    testPred = poisson_result.predict(testX.T)

    train_score = r2_score(trainY, trainPred)

    test_score = r2_score(testY, testPred)

    res = {'intercept': intercept, 'coeff': coeff, 'test_score': test_score, 'train_score': train_score, 'DE': DE, 'AIC': AIC, 'BIC': BIC, 'fitted_vals': fitted_vals, 'chi2': chi2, 'pvalues': pvalues, 'tvalues': tvalues}

    res_data = {'testPred': testPred, 'model': poisson_model, 'result_obj':poisson_result}

    inp = {'trainX': trainX, 'trainY': trainY, 'testX': testX, 'testY': testY}

    return res, res_data, inp

def agg_fit_glm(study):

    agg_res_dict = {}

    agg_event_times, agg_event_objects, agg_events_binary, time_index = aggregate_event_times(study)

    synth_regs = agg_synth_regressors(time_index)

    bio_regs = agg_bio_regressors(study)
    
    bio_regs = adjust_bio_regressors_to_cts(np.array(bio_regs), agg_events_binary.shape[1])

    agg_cell_ct = 0
    animal_ct = 0

    for animal in study.animals:
        agg_res_dict[animal.animal_id] = {}
        ses_ct = 0
        for key in animal.ensembles.keys():
            ensemble = animal.ensembles[key]
            agg_res_dict[animal.animal_id][key] = {}
            for i in range(len(ensemble.cells)):
                cell = ensemble.cells
                agg_res_dict[animal.animal_id][key][i] = {}

                X = synth_regs
                
                # X = np.array(bio_regs[animal_ct,ses_ct,:,:])

                # # check_collinearity(X)

                # # if X.shape[1] != synth_regs.shape[1]:
                # length = min(X.shape[1], synth_regs.shape[1])
                # X = X[:, :length]
                # # synth_regs = synth_regs[:,:length]
                # # X = np.vstack((X[:,:], synth_regs[:,:]))


                # X = normalize_regs(X)

                # print('PRE' + str(X.shape), len(time_index))

                # design_matrices = make_design_from_regs(X, time_index, 4)
                # print('POST' + str(design_matrices.shape))
                # design_matrices[design_matrices != design_matrices] = 0
                y = agg_events_binary[agg_cell_ct]

                # # print(design_matrices.shape, y.shape)
                # y = y[:length]


                ############ TESTING NEED TO REMOVE ############
                # y = y[:design_matrices.shape[1]]

                # # plt.imshow(design_matrices)
                # print(design_matrices.shape, y.shape)

                res, res_data, inp = fit_glm_statsmodel(y, X, regularized=False)
                # res, res_data, inp = fit_glm_sklearn(y, design_matrices)

                # for k in range(design_matrices.shape[1]):
                #     trial_design = design_matrices[:,k,:]



                # for i in range(len(X)):
                #     res, res_data, inp = fit_glm(y, X[i])

                agg_res_dict[animal.animal_id][key][i]['res'] = res
                agg_res_dict[animal.animal_id][key][i]['res_data'] = res_data
                agg_res_dict[animal.animal_id][key][i]['inp'] = inp

                agg_cell_ct += 1
            ses_ct += 1
        animal_ct += 1

    return agg_res_dict

def make_design_from_regs(X, time_index, trial_count):
    window = 5 # seconds

    dt = time_index[1] - time_index[0]
    
    lags = int(window/dt)

    length = len(time_index)
    # length = int(60/dt)


    ############ TESTING NEED TO REMOVE ############
    # X = X[0:1]
    # trial_count = 1

    # print(X.shape)

    # print(length, lags)

    design_matrices = []
    for i in range(X.shape[0]):
        # trial_design = []
        # for k in range(trial_count):
        #     trial = X[i,k*length:(k+1)*length]

        paddedStim = np.hstack((np.zeros((lags)), X[i,0:length]))
        Xdsgn = np.zeros((length,lags)) 
        for j in range(0,paddedStim.shape[0]-lags):
            # print(j, lags, paddedStim.shape, paddedStim[j:j+lags].shape)
            # print(paddedStim[j:j+lags].shape, lags)
            Xdsgn[j] = paddedStim[j:j+lags] 
            
            # trial_design.append(Xdsgn)
            design_matrices.append(Xdsgn)
        # design_matrices.append(trial_design)
    
    return np.array(design_matrices).reshape((X.shape[0]*trial_count*lags,-1)).T



In [10]:
agg_res_dict = agg_fit_glm(study)

C:\Users\aaoun\AppData\Local\Temp\ipykernel_9188\18825785.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return agg_events, np.array(agg_event_objects), np.array(agg_events_binary), np.array(time_index)


KeyboardInterrupt: 

In [ ]:
c = 0
for animal in agg_res_dict:
    for session in agg_res_dict[animal]:
        for cell in agg_res_dict[animal][session]:
            print('train: ' + str(agg_res_dict[animal][session][cell]['res']['DE']*100) + ' test: ' + str(agg_res_dict[animal][session][cell]['res']['test_score']*100))
            # if c== 0:
            # plt.plot(agg_res_dict[animal][session][cell]['res']['coeff'])
            xtest = agg_res_dict[animal][session][cell]['inp']['testX']
            xtrue = agg_res_dict[animal][session][cell]['inp']['testY']
            trainX = agg_res_dict[animal][session][cell]['inp']['trainX']
            trainY = agg_res_dict[animal][session][cell]['inp']['trainY']
            
            b = agg_res_dict[animal][session][cell]['res']['coeff']
            xpred = np.exp(np.dot(trainX[1:,:].T, b)) + trainX[0,:] * agg_res_dict[animal][session][cell]['res']['intercept']
            # plt.plot(b[:500])
            # print(xtest.shape, xtrue.shape, xpred.shape, trainX.shape, trainY.shape)
            # if c == 1:
            #     ax = plt.subplot(1,1,1)
            #     ax.plot(trainY)
            #     ax2 = ax.twinx()
            #     ax2.plot(xpred, color='r')
            #     plt.show()
            c += 1

In [25]:

agg_event_times, agg_event_objects, agg_events_binary, time_index = aggregate_event_times_matched(study)


4
5
4
4
4
4
4


C:\Users\aaoun\AppData\Local\Temp\ipykernel_6092\2641687853.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return agg_events, np.array(agg_event_objects), np.array(agg_events_binary), np.array(time_index)


In [33]:
len(agg_event_times[7])

23

In [9]:
# Cell spike times
agg_event_times, agg_event_objects, agg_events_binary, time_index = aggregate_event_times(study)

synth_regs = agg_synth_regressors(time_index)

bio_regs = agg_bio_regressors(study)

bio_regs = adjust_bio_regressors_to_cts(bio_regs, agg_events_binary.shape[1])


C:\Users\aaoun\AppData\Local\Temp\ipykernel_6092\18825785.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return agg_events, np.array(agg_event_objects), np.array(agg_events_binary), np.array(time_index)


IndexError: tuple index out of range

In [35]:
len(bio_regs)

AttributeError: 'list' object has no attribute 'shape'